In [70]:
import pandas as pd
import geopandas as gp
import numpy as np
import os
import requests
import urllib.parse

In [71]:
# initiate file paths
input_file_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/aggregated_results_geom.csv"

In [105]:
def drop_levels(df):
    df.reset_index(inplace = True)
    df.columns = df.columns.droplevel(1)
    return df

In [72]:
all_results = pd.read_csv(input_file_path)

In [73]:
sl_by_group = all_results.copy()

# count screening level exceedances by pah, pcb, df, and each individual metal
sl_by_group['Chemical Group'] = np.where(sl_by_group['Chemical Group']=='RCRA8', sl_by_group['Chemical'], sl_by_group['Chemical Group'])

In [74]:
# count of total exceedances by group
cnt_exceedances = sl_by_group[sl_by_group['SL_exceeded']=='Y']
cnt_exceedances = cnt_exceedances.groupby(by = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude']).agg({'SL_exceeded': ['count']}).reset_index()
cnt_exceedances = drop_levels(cnt_exceedances)

In [75]:
# calculate total screening level count
cnt_sl = sl_by_group[sl_by_group['SL_exceeded'] != 'No Screening Level Identified']
cnt_sl = cnt_sl.groupby(by = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude']).agg({'SL_exceeded': ['count']}).reset_index()
cnt_sl = drop_levels(cnt_sl)

In [77]:
cnt_exceedances_merge = cnt_exceedances.merge(cnt_sl, left_on = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude'], right_on = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude'], how = 'left')

/tmp/ipykernel_1384/3864438281.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  cnt_exceedances_merge = cnt_exceedances.merge(cnt_sl, left_on = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude'], right_on = ['DATE','Sample ID','Chemical Group','Medium','Latitude','Longitude'], how = 'left')


In [102]:
cnt_exceedances_merge.rename(columns = {'SL_exceeded_x':'cnt_sl_exceed', 'SL_exceeded_y':'tot_cnt_sl'}, inplace = True)
cnt_exceedances_merge['pct_exceed'] = cnt_exceedances_merge['cnt_sl_exceed'] /cnt_exceedances_merge['tot_cnt_sl']

CALCULATE MOST STRINGENT

In [107]:
stringent = all_results.groupby(by =['Medium', 'Chemical Group', 'Chemical', 'Scenario']).agg({'Screening Level Measurement': ['min']}).reset_index()
stringent = drop_levels(stringent)

In [117]:
only_results = all_results[['DATE','Sample ID', 'Medium', 'Chemical Group', 'Chemical','Result Value']].drop_duplicates()

In [119]:
# merge 
stringent_merge = only_results.merge(stringent, left_on = ['Medium', 'Chemical Group', 'Chemical'], right_on = ['Medium', 'Chemical Group', 'Chemical'], how = 'inner')

In [120]:
stringent_merge['SL_exceed'] = np.where(stringent_merge_exceed['Screening Level Measurement'] < stringent_merge_exceed['Result Value'], 'Y','N')
stringent_merge['SL_diff'] = stringent_merge['Screening Level Measurement'] - stringent_merge_exceed['Result Value']

TypeError: '<' not supported between instances of 'str' and 'float'

EXPORT FOR MAPS

In [61]:
# export results that only exceed most stringent screening levels
stringent_merge_exceed.to_csv("/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/stringent_exceeded.csv", index = False)

In [62]:
# export results based on count of exceedances and for each results
cnt_exceedances_merge.to_csv("/home/nweiss/gdrive/Year 2/Summer - Duwamish/Sampling_Results/count_exceedances.csv", index = False)